# PyCity Schools Analysis

* This report summarises the area-wide standardised test results for the schools within the local government area as well as school budget, and size data. The purpose of this report is to draw any observable trends from the data displayed, to help inform future decisions surrounding the budgets and priorities. This report summarises the data through filtering by grade, per student capita, school type, and school size, alongside the average maths and reading scores, and the passing percentages of the student population.

* The data displayed in reference to school scores by school size suggests that schools with a smaller student population (<1000) achieve higher test results in all areas than schools with a larger student population (2000-5000). This may be due to a number of factors one of which being class sizes, resulting in lower teacher to student ratios in schools with a smaller student population which would potentially allow for more student support during class hours, and less assistance for students in larger class sizes in schools with larger student populations. Additionally due to a smaller student population, teachers will likely experience reduced workloads as they aren't required to manage as many students, resulting in consistency in delivering better outcomes. 

* The data displayed in reference to school type suggests that students from Independent schools achiever higher test results in all areas than students from Government schools. Comparing to the above analysis suggesting that schools with a smaller student population achieve higher test results, we can see that most Independent schools in the data provided have smaller student populations than the government schools in the dataset. The potential reason for this may be due to the increased tuition fees Independent schools receive, which can often lead to fewer opportunities available for student enrolment at Independent schools for students from lower socioeconomic backgrounds. This means that Independent schools can be more selective in accepting potential incoming students which could lead to better outcomes for the whole population.

* In conclusion the above analysis provides a minor overview of the potential correlations between students achieving higher test results and student population size and school type. However due to several limitations within this dataset, additional data would be required in order to confirm the previously detailed trends. Suggested reports include collecting data regarding class sizes, tuition fees, and the socioeconomic status of students and schools.
---

In [31]:
# Import Dependencies 
from pathlib import Path
import pandas as pd

In [33]:
# Load files
schools_complete = Path("Resources/schools_complete.csv")
students_complete = Path("Resources/students_complete.csv")

In [35]:
# Read Data into DataFrames
school_data = pd.read_csv(schools_complete)
student_data = pd.read_csv(students_complete)

In [37]:
# Combine into single dataset
school_student_complete = pd.merge(student_data, school_data, how="left", on="school_name")

# Local Government Area Summary

In [40]:
# Calculate the Totals (Schools and Students)
school_count = len(school_data["School ID"].unique())
student_count = len(student_data["Student ID"].unique())

# Calculate the Total Budget
total_budget = school_data["budget"].sum()

In [42]:
# Calculate the Average Scores
average_maths_score = student_data["maths_score"].mean()
average_reading_score = student_data["reading_score"].mean()

In [44]:
# Calculate the Percentage Pass Rates
passing_maths_count = school_student_complete[(school_student_complete["maths_score"] >= 50)].count()["student_name"]
passing_maths_percentage = passing_maths_count / float(student_count) * 100
passing_reading_count = school_student_complete[(school_student_complete["reading_score"] >= 50)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

passing_maths_reading_count = school_student_complete[(school_student_complete["maths_score"] >= 50) & (school_student_complete["reading_score"] >= 50)].count()["student_name"]
overall_passing_rate = passing_maths_reading_count / float(student_count) * 100

In [46]:
# Convert to DataFrame
area_summary = pd.DataFrame({"Total Schools": [school_count],
                             "Total Students": student_count,
                             "Total Budget": total_budget,
                             "Average Maths Score": average_maths_score,
                             "Average Reading Score": average_reading_score,
                             "% Passing Maths": passing_maths_percentage,
                             "% Passing Reading": passing_reading_percentage,
                             "% Overall Passing": overall_passing_rate})

# Formatting
area_summary["Total Students"] = area_summary["Total Students"].map("{:,}".format)
area_summary["Total Budget"] = area_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
area_summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


# School Summary

In [49]:
# Use the code provided to select the type per school from school_data
school_types = school_data.set_index(["school_name"])["type"]

# Calculate the total student count per school from school_data
per_school_counts = school_data.set_index(["school_name"])["size"]

# Calculate the total school budget and per capita spending per school from school_data
per_school_budget = school_data.set_index(["school_name"])["budget"]
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores per school from school_data_complete
per_school_maths = school_student_complete.groupby(["school_name"])["maths_score"].mean()
per_school_reading = school_student_complete.groupby(["school_name"])["reading_score"].mean()

In [51]:
# Get the students who passed maths and passed reading by creating separate filtered DataFrames from school_data_complete.
school_passing_maths = school_student_complete[(school_student_complete["maths_score"] >= 50)]
school_passing_reading = school_student_complete[(school_student_complete["reading_score"] >= 50)]

# Get the students who passed both reading and maths in a separate DataFrame from school_data_complete.
passing_maths_and_reading = school_student_complete[(school_student_complete["maths_score"] >= 50) & (school_student_complete["reading_score"] >= 50)]


In [53]:
#  Calculate the Percentage Pass Rates
per_school_passing_maths = school_passing_maths.groupby(["school_name"]).count()["Student ID"] / school_student_complete.groupby(["school_name"]).count()["Student ID"] * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["Student ID"] / school_student_complete.groupby(["school_name"]).count()["Student ID"] * 100
overall_passing_rate = passing_maths_and_reading.groupby(["school_name"]).count()["Student ID"] / school_student_complete.groupby(["school_name"]).count()["Student ID"] * 100

In [80]:
# Convert to DataFrame
per_school_summary = pd.DataFrame({"School Type": school_types,
                                   "Total Students": per_school_counts,
                                   "Total School Budget": per_school_budget,
                                   "Per Student Budget": per_school_capita,
                                   "Average Maths Score": per_school_maths,
                                   "Average Reading Score": per_school_reading,
                                  "% Passing Maths": per_school_passing_maths,
                                   "% Passing Reading": per_school_passing_reading,
                                   "% Overall Passing": overall_passing_rate})

# Creating a copy of school summary data prior to formatting to allow for number comparisons
school_spending_df = per_school_summary.copy()

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766


# Top Performing Schools (By % Overall Passing)

In [58]:
# Sort and show top five schools
top_schools = per_school_summary.sort_values(by=["% Overall Passing"], ascending = False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


# Bottom Performing Schools (By % Overall Passing)

In [61]:
# Sort and show bottom five schools
bottom_schools = per_school_summary.sort_values(by=["% Overall Passing"])
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


# Maths Scores by Year

In [64]:
# Create data series of scores by year levels using conditionals
year_nine = school_student_complete[(school_student_complete["year"] == 9)]
year_ten = school_student_complete[(school_student_complete["year"] == 10)]
year_eleven = school_student_complete[(school_student_complete["year"] == 11)]
year_twelve = school_student_complete[(school_student_complete["year"] == 12)]

# Group each by school name
year_nine_scores = year_nine.groupby(["school_name"])["maths_score"].mean().apply(pd.Series)
year_ten_scores = year_ten.groupby(["school_name"])["maths_score"].mean().apply(pd.Series)
year_eleven_scores = year_eleven.groupby(["school_name"])["maths_score"].mean().apply(pd.Series)
year_twelve_scores = year_twelve.groupby(["school_name"])["maths_score"].mean().apply(pd.Series)

# Combine series into single DataFrame
maths_scores_by_year = pd.concat([year_nine_scores, year_ten_scores, year_eleven_scores, year_twelve_scores], axis = 1)
maths_scores_by_year.columns = ["Year 9", "Year 10", "Year 11", "Year 12"]

# Minor data wrangling
maths_scores_by_year.index.name = None

# Display the DataFrame
maths_scores_by_year

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


# Reading Score by Year

In [67]:
# Create data series of scores by year levels using conditionals
year_nine = school_student_complete[(school_student_complete["year"] == 9)]
year_ten = school_student_complete[(school_student_complete["year"] == 10)]
year_eleven = school_student_complete[(school_student_complete["year"] == 11)]
year_twelve = school_student_complete[(school_student_complete["year"] == 12)]

# Group each by school name
year_nine_scores = year_nine.groupby(["school_name"])["reading_score"].mean().apply(pd.Series)
year_ten_scores = year_ten.groupby(["school_name"])["reading_score"].mean().apply(pd.Series)
year_eleven_scores = year_eleven.groupby(["school_name"])["reading_score"].mean().apply(pd.Series)
year_twelve_scores = year_twelve.groupby(["school_name"])["reading_score"].mean().apply(pd.Series)

# Combine series into single DataFrame
reading_scores_by_year = pd.concat([year_nine_scores, year_ten_scores, year_eleven_scores, year_twelve_scores], axis = 1)
reading_scores_by_year.columns = ["Year 9", "Year 10", "Year 11", "Year 12"]

# Minor data wrangling
reading_scores_by_year.index.name = None

# Display the DataFrame
reading_scores_by_year

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


# Scores by School Spending

In [70]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

In [86]:
# Use `pd.cut` on the per_school_capita Series from earlier to categorise per student spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"].astype(int),
                                                             spending_bins, labels=group_names,
                                                             include_lowest=True)
# Formatting
school_spending_df["Total School Budget"] = school_spending_df["Total School Budget"].map("${:,.2f}".format)
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].map("${:,.2f}".format)

# Convert Spending Ranges (Per Student) to a string
school_spending_df["Spending Ranges (Per Student)"] = school_spending_df["Spending Ranges (Per Student)"]
school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405,$585-630
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791,<$585
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051,$630-645
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880,$630-645
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150,$585-630
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617,$645-680
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717,<$585
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376,$645-680
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766,$645-680


In [94]:
#  Calculate averages for the desired columns.
spending_maths_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"],observed=True)["Average Maths Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"],observed=True)["Average Reading Score"].mean()
spending_passing_maths = school_spending_df.groupby(["Spending Ranges (Per Student)"],observed=True)["% Passing Maths"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"],observed=True)["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"],observed=True)["% Overall Passing"].mean()

In [104]:
# Assemble into DataFrame
spending_summary = pd.concat([spending_maths_scores, spending_reading_scores, spending_passing_maths, spending_passing_reading, overall_passing_spending], axis = 1)
spending_summary.columns = ["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]

# Display results in descending order
spending_summary = spending_summary.sort_values(by=["Spending Ranges (Per Student)"], ascending = False)
spending_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
$645-680,68.884391,69.045403,81.568470,81.769716,66.756253
$630-645,69.854807,69.838814,84.686139,83.763585,71.004977
$585-630,72.065868,71.031297,91.518824,87.292423,79.876293
<$585,71.364587,70.716577,88.835926,86.390517,76.721458


# Scores by School Size

In [107]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [113]:
# Create a copy of the school summary for later aggregations
# This step can be skipped but it's best to make a copy.
school_size_df = school_spending_df.copy()

In [115]:
# Use `pd.cut` on the per_school_counts Series from earlier to categorise school size based on the bins.
school_size_df["School Size"] = pd.cut(school_size_df["Total Students"].astype(int),
                                                             size_bins, labels=group_names,
                                                             include_lowest=True)

# Convert School Size to a string
school_size_df["School Size"] = school_size_df["School Size"].astype(str)
school_size_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
school_name,,,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405,$585-630,Large (2000-5000)
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791,<$585,Medium (1000-2000)
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051,$630-645,Large (2000-5000)
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880,$630-645,Large (2000-5000)
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150,$585-630,Medium (1000-2000)
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617,$645-680,Large (2000-5000)
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717,<$585,Small (<1000)
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376,$645-680,Large (2000-5000)
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766,$645-680,Large (2000-5000)


In [117]:
# Calculate averages for the desired columns.
size_maths_scores = school_size_df.groupby(["School Size"],observed=True)["Average Maths Score"].mean()
size_reading_scores = school_size_df.groupby(["School Size"],observed=True)["Average Reading Score"].mean()
size_passing_maths = school_size_df.groupby(["School Size"],observed=True)["% Passing Maths"].mean()
size_passing_reading = school_size_df.groupby(["School Size"],observed=True)["% Passing Reading"].mean()
size_overall_passing = school_size_df.groupby(["School Size"],observed=True)["% Overall Passing"].mean()

In [119]:
# Assemble into DataFrame
size_summary = pd.concat([size_maths_scores, size_reading_scores, size_passing_maths, size_passing_reading, size_overall_passing], axis = 1)
spending_summary.columns = ["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]

# Display results
size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348


# Scores by School Type

In [122]:
# Create new series using groupby for:
# Type | Average Maths Score | Average Reading Score | % Passing Maths | % Passing Reading | % Overall Passing

type_maths_scores = per_school_summary.groupby(["School Type"],observed=True)["Average Maths Score"].mean()
type_reading_scores = per_school_summary.groupby(["School Type"],observed=True)["Average Reading Score"].mean()
type_passing_maths = per_school_summary.groupby(["School Type"],observed=True)["% Passing Maths"].mean()
type_passing_reading = per_school_summary.groupby(["School Type"],observed=True)["% Passing Reading"].mean()
type_overall_passing = per_school_summary.groupby(["School Type"],observed=True)["% Overall Passing"].mean()

In [124]:
# Assemble into DataFrame
type_summary = pd.concat([type_maths_scores, type_reading_scores, type_passing_maths, type_passing_reading, type_overall_passing], axis = 1)
spending_summary.columns = ["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]

# Display results
type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
